## Load packages

In [157]:
import subprocess, msprime, tskit, os, statistics, pyslim
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import math
import os
import pickle

## Write input files

In [158]:
def WriteInput(ts, Nrun, SampleSize, Ne, Stop, SeqLen, MutRate, RecRate, seed1, seed2, filename):
    filename_tmp = filename + "1"
    f1 = open(filename_tmp,"w+")
    f1.write("Number of runs = %d\n" % Nrun)
    f1.write("Effective population size = %d\n" % Ne)
    f1.write("Recombination = %.15f\n" % RecRate)
    f1.write("Mutation = %.15f\n" % MutRate)
    f1.write("Length = %f\n" % SeqLen)
    f1.write("Stop = %d\n" % Stop)
    f1.write("Fix seed = %d, %d\n" % (seed1, seed2))
    f1.write("#")
    f1.close()
    
    filename_tmp = filename + "2"
    f2 = open(filename_tmp,"w+")
    
    # Get distinct haplotypes
    geno =  ts.genotype_matrix() 
    L = len(geno)
    f2.write("Loci = %d\n" % (L+1))
    SampleSize = ts.get_sample_size()
    f2.write("Genes = %d\n" % SampleSize)
          
    # initialise with the first distinct haplotype
    haps_1 = [0]
    haps_2 = [geno[i][0] for i in range(L)]
    haps = np.hstack((haps_1, haps_2))
    # the number of this haplotype is 1
    #haps.append(1)
    haps = np.hstack((haps, [1]))
    haps = np.array([haps])
    ntype = 1
    order = [0]

    for j in range(1,SampleSize):
        new_1 = [0]
        new_2 = [geno[i][j] for i in range(L)]
        new = np.hstack((new_1, new_2))
        flag = 0 # flag=0: a new distinct haplotype
        for k in range(ntype):
            if(haps[k][:(L+1)].tolist() == new.tolist()):
                haps[k][(L+1)] += 1
                flag = 1
                order.append(j)
                break
        if flag == 0:
            order.append(ntype)
            ntype += 1
            #new.append(1)
            new = np.hstack((new, [1]))
            haps = np.vstack([haps, new]) 
          
    f2.write("Distinct Haplotypes = %d\n" % ntype)
    tot=0
    for i in range(ntype):
        tot = tot+ np.sum(haps[i,:(L+1)])*haps[i,(L+1)]
    stationary = tot/(L*SampleSize)
    f2.write("Stationary = %.5f\n" % (1-stationary))
    
    f2.write("Haplotypes\n")
    for i in range(haps.shape[0]):
        for j in range(haps.shape[1]):
            if j == L+1:
                f2.write("%d\n" % haps[i][j])
            else:
                f2.write("%d " % haps[i][j])
    
    # Mutation transition rates
    f2.write("\nMutation\n0.0 1.0 0.0 1.0\n")
             
    # Genetic positions
    f2.write("\nPositions\n")
    positions = [0.0]
    for s in ts.sites():
        positions.append(s.position)
    for i in range(len(positions)):
        f2.write("%.2f " %(positions[i]))
             
    f2.write("\n#")
    f2.close()
    order = sorted(range(len(order)), key=lambda k: order[k])
    return order, L, 1-stationary;

## Set parameters

In [156]:
Ne = 775
SampleSize = 20
Nrun = 5000 # number of SIS runs
Stop = 11 # the stopping criteria
SeqLen = 16569 # sequence length
MutRate = 2.6e-7 # mutation rate
RecRate = 0 # recombination rate
seed1 = 1 # seed1 and seed2 are used in our C progra m
seed2 = 10
seed = 123
filename = "C:\\Users\\blabl\\Desktop\\git repos\\Sampling_Wu\\infile"

## Sample case and control sequences
To sample case sequences, the easier way is to find nodes that contribute to exactly ten sequences at present. The ten sequences at present are set to case sequences. If such nodes are not enough, I look for nodes that contribute to more than ten sequences at present, and then randomly select ten sequences out of the present-time-children of such nodes. In this example, there are eight nodes that contribute to exactly ten sequences at present.

In [145]:
tree = pyslim.load("C:\\Users\\blabl\\Dropbox\\RA_project\\SliM\\recipe_new.trees")
tot = 0
for c in range(Ne*2):
    clade1 = c
    count = 0
    ts = tree.at(100)
    for s in range(Ne*2, Ne*4):
        if ts.is_descendant(s, clade1):
            count = count + 1
    if count>0:
        tot = tot + count
        if count == 10:
            print("clade %d contribute to %d samples" %(c, count))

clade 126 contribute to 10 samples
clade 210 contribute to 10 samples
clade 285 contribute to 10 samples
clade 294 contribute to 10 samples
clade 417 contribute to 10 samples
clade 448 contribute to 10 samples
clade 779 contribute to 10 samples
clade 1146 contribute to 10 samples


In [159]:
roots = [126] # node 126, 210, 285, 294, 417, 448, 779 and 1146 contribute to ten sequences at present
s1 = []

ts = tree.at(100)
for s in range(Ne*2, Ne*4):
    if ts.is_descendant(s, roots[0]):
        s1.append(s)
cases = s1
cases

[1721, 1867, 1880, 1882, 1925, 2130, 2169, 2338, 2386, 2466]

In [160]:
np.random.seed(18)
rest = [x for x in range(Ne*2, Ne*4) if x not in cases]
control = np.random.choice(rest, size=(SampleSize - len(cases)), replace=False)
control

array([2930, 2459, 2010, 2627, 1679, 2579, 3082, 2632, 2854, 2442])

## Get the tree for the selected sample sequences
1. Do recapitation to the original SLiM tree
2. Simplify the recapitated tree so the simplified tree only contians history of the selected sample sequences
3. Add mutations to the simplified tree
4. Get the haplotypes that will be used to write the input files for the C program

In [161]:
tree_re = pyslim.recapitate(tree, ancestral_Ne=Ne, recombination_rate=0, random_seed=seed)
com = np.hstack([cases, control])
tree_com = tree_re.simplify(samples=com, keep_unary=True, map_nodes=True)
map_node = tree_com[1]
tree_com = msprime.sim_mutations(tree_com[0], rate=MutRate, random_seed=seed, keep=True)
geno_com = tree_com.genotype_matrix()

In [162]:
np.shape(geno_com) # 26 segregating sites, 20 sequences

(26, 20)

In [163]:
for var in tree_com.variants():
    print(var.site.position, var.alleles, var.genotypes, sep="\t")

68.0	('C', 'T')	[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
712.0	('A', 'G')	[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0]
1378.0	('T', 'A')	[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
1798.0	('C', 'G')	[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
3200.0	('G', 'C')	[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 0 1 1]
3618.0	('G', 'C')	[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]
3832.0	('G', 'A')	[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
4482.0	('A', 'T')	[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
4652.0	('T', 'C')	[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
4866.0	('T', 'C')	[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 0 1 1]
5134.0	('A', 'G')	[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0]
5248.0	('C', 'G')	[0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 1 1 0 0]
6095.0	('G', 'A')	[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]
6299.0	('T', 'A')	[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
6376.0	('T', 'C')	[0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 1 1 0 0]
7625.0	('C', 'T')	[1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 0 1 1]
7904.0	('G', 'C')	[0 0 0 0 0 0 0 0 0 0 0 0 

In [164]:
# store the ID of the MRCA of case clade in the simplified tree 
new_case_MRCA = map_node[roots[0]]

In [165]:
# check if the samples are descendants of their MRCA
tree_com.at(100).is_descendant(0, new_case_MRCA)

True

## Write the input files for the C program

In [166]:
order_com = WriteInput(tree_com, Nrun, SampleSize, Ne, Stop, SeqLen, MutRate, RecRate, seed1, seed2, filename)

## Count the number of mutations before MRCA of case clade is found

In [167]:
# get the node time of the case clade's MRCA
case_MRCA_time = tree_com.tables.nodes[new_case_MRCA].time

In [168]:
sum(tree_com.tables.mutations.time < case_MRCA_time)

0